<a href="https://colab.research.google.com/github/laribferraz/chatbot_assistente_culinario/blob/main/Chatbot_Assistente_Culin%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 - Instalação e Importações

In [36]:
!pip install -q -U google-generativeai requests
!pip install googletrans==4.0.0-rc1
!pip install langdetect python-dotenv
!pip install google-cloud-translate
!pip install --upgrade google-generativeai
!pip install python-dotenv

import os
import json
import random
import requests
from typing import List
import time

import google.generativeai as genai
from googletrans import Translator
from langdetect import detect
from dotenv import load_dotenv
from google.cloud import translate_v3beta1 as translate

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()


False

2 - Configurações

In [37]:
# Configurações
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

generation_config = {
    "candidate_count": 1,
    "temperature": 0.7,
}

model = genai.GenerativeModel(
    model_name='gemini-pro',
    generation_config=generation_config,
)

chat = model.start_chat()

EDAMAM_APP_ID = os.getenv("EDAMAM_APP_ID")
EDAMAM_APP_KEY = os.getenv("EDAMAM_APP_KEY")

translator = Translator()
translate_client = translate.TranslationServiceClient()

3 - Funções Auxiliares

In [34]:
def buscar_receita(ingredientes: str, restricoes: str = "") -> dict:
    url = f"https://api.edamam.com/search?q={ingredientes}&app_id={EDAMAM_APP_ID}&app_key={EDAMAM_APP_KEY}&from=0&to=3{restricoes}"
    response = requests.get(url)
    data = response.json()
    return data['hits'][0]['recipe'] if 'hits' in data and data['hits'] else None


def formatar_receita(receita: dict, porcoes: int = None) -> str:
    if not receita:
        return "Desculpe, não encontrei nenhuma receita com esses ingredientes."

    if porcoes:
        receita_ajustada = {
            "label": receita['label'],
            "ingredientLines": [f"{i} (ajuste para {porcoes} porções)" for i in receita['ingredientLines']],
            "url": receita['url'],
            "calories": receita.get('calories', 'N/A')
        }
        return f"""## {receita_ajustada['label']} (para {porcoes} porções)

**Ingredientes:**
{', '.join(receita_ajustada['ingredientLines'])}

**Instruções:**
Para as instruções completas, acesse: {receita_ajustada['url']}

**Informações Nutricionais:**
{receita_ajustada.get('calories', 'N/A')} calorias (aproximado)
"""

    return f"""## {receita['label']}

**Ingredientes:**
{', '.join(receita['ingredientLines'])}

**Instruções:**
Para as instruções completas, acesse: {receita['url']}

**Informações Nutricionais:**
{receita.get('calories', 'N/A')} calorias
"""


def gerar_piada_culinaria() -> str:
    piadas = [
        "Por que o tomate foi ao banco? Para pegar extrato de tomate!",
        "Qual a fruta que os fantasmas mais gostam? Morango!",
        "Qual é o legume que gosta de música? A beterraba!"
    ]
    return random.choice(piadas)


# --- Funções para Tradução e Processamento de Prompt ---

def detectar_idioma(texto: str) -> str:
    return detect(texto)


def traduzir_texto(texto: str, idioma_destino: str = 'en') -> str:
    response = translate_client.translate_text(
        request={
            "parent": f"projects/your-project-id/locations/global",  # Substitua pelo seu ID de projeto
            "contents": [texto],
            "mime_type": "text/plain",
            "target_language_code": idioma_destino
        }
    )
    return response.translations[0].translated_text

def processar_prompt(prompt: str, receita_atual: dict = None) -> str:
    idioma = detectar_idioma(prompt)
    if idioma != 'en':
        prompt = traduzir_texto(prompt, 'en')

    if receita_atual:
        try:
            prompt = f"Contexto da receita:\n{json.dumps(receita_atual)}\n\nPrompt do usuário:\n{prompt}"
        except TypeError:
            print("Aviso: Erro ao serializar receita_atual em JSON.")

    return prompt, idioma


# --- Função para Simular Streaming ---

def print_streaming_response(response: str) -> None: # Recebe a resposta completa como string
    words = response.split() # Divide a resposta em palavras
    for word in words:
        print(word, end=" ", flush=True)
        time.sleep(0.05) # Pausa para simular o streaming
    print()

4 - Variáveis de Contexto e Inicialização

In [22]:
restricoes = ""
receita_atual = None
lista_compras: List[str] = []
historico_conversa: List[str] = []

print("Olá! Sou seu assistente de culinária. O que você gostaria de cozinhar hoje?")

Olá! Sou seu assistente de culinária. O que você gostaria de cozinhar hoje?


5 - Loop Principal do Chatbot

In [ ]:
while True:
    print("\nPara começar, escolha uma opção:")
    print("1. Encontrar uma receita")
    print("2. Adicionar item na lista de compras")
    print("3. Mostrar minha lista de compras")
    print("4. Contar uma piada culinária")
    print("5. Sair")

    opcao = input("Digite o número da opção desejada: ")

    if opcao == '1':
        prompt = input("Descreva a receita que você deseja encontrar: ")
        if "want to make" in prompt.lower() or "recipe with" in prompt.lower():
            comando = prompt.lower().split(" ")
            ingredientes = comando[comando.index("with")+1:]
            porcoes = int(comando[comando.index("for") + 1]) if "for" in comando else None
            restricoes = f"&health={prompt.lower().split('without')[1].strip()}" if "without" in prompt.lower() else ""
            receita_atual = buscar_receita(ingredientes, restricoes)
            resposta = formatar_receita(receita_atual, porcoes)
            print(resposta)

    elif opcao == '2':
        prompt = input("Digite o item que deseja adicionar na lista de compras: ")
        if "adicionar" in prompt.lower() and "lista de compras" in prompt.lower():
            item = prompt.lower().split("adicionar")[1].split("na lista de compras")[0].strip()
            lista_compras.append(item)
            print(f"{item} adicionado à lista de compras!")

    elif opcao == '3':
        if lista_compras:
            print("Sua lista de compras:")
            for item in lista_compras:
                print(f"- {item}")
        else:
            print("Sua lista de compras está vazia!")

    elif opcao == '4':
        print(gerar_piada_culinaria())

    elif opcao == '5':
        print("Bom apetite! Até a próxima.")
        break

    else:
        print("Opção inválida. Tente novamente.")
        continue

    # --- Interação com o Gemini ---

    if opcao in ('1', '2', '3', '4'):
        prompt, idioma = processar_prompt(prompt, receita_atual)

        # Simulação de Streaming
        response = chat.send_message(prompt)
        print_streaming_response(response.text) # Passa o texto completo para a função

        if idioma != 'en':
            texto_traduzido = traduzir_texto(response.text, idioma) # Traduz a resposta completa
            print(texto_traduzido)

print("Bom apetite! Até a próxima.")


Para começar, escolha uma opção:
1. Encontrar uma receita
2. Adicionar item na lista de compras
3. Mostrar minha lista de compras
4. Contar uma piada culinária
5. Sair
